In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
import load_ship_data as lsd

In [2]:
np.random.seed(522)
data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)    
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(len(Xvalid.shape))
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
print(Xtrain.shape)
print(ytrain.shape)

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600
[0.7525 0.2475]
(2800, 19200)
(600, 19200)
2
(3400, 19200)
(3400,)


In [9]:
class Fld:
    def __init__(self):
        self.class_means={}
        self.class_scatter={}
        
    def fit(self,Xtrain,ytrain):
        data = np.column_stack((Xtrain, ytrain))
        class0=data[np.where(data[:,-1]==0)]
        class1=data[np.where(data[:,-1]==1)]
        
        self.class_means[0]=np.mean(class0[:,:-1],axis=0)
        self.class_means[1]=np.mean(class1[:,:-1],axis=0)
        self.class_scatter[0]=np.dot((class0[:,:-1]-self.class_means[0]).T,(class0[:,:-1]-self.class_means[0]))
        self.class_scatter[1]=np.dot((class1[:,:-1]-self.class_means[1]).T,(class1[:,:-1]-self.class_means[1]))
        self.scatter=np.linalg.inv(self.class_scatter[0]+self.class_scatter[1])
        
        temp=self.class_means[0]-self.class_means[1]
        self.w=np.dot(self.scatter,temp)
       # print(self.w)
        projected=np.dot(self.w,Xtrain.T)
       # print(projected.shape)
        data=np.c_[projected,ytrain]
   #     print(data)
        class0=data[np.where(data[:,-1]==0)]
        class1=data[np.where(data[:,-1]==1)]
        self.mean0=np.mean(class0[:,0],axis=0)
        self.mean1=np.mean(class1[:,0],axis=0)
        self.var0=np.var(class0[:,0],axis=0)
        self.var1=np.var(class1[:,0],axis=0)
    #    print(self.var0,self.mean0)
     #   print(projected.shape)
        return projected
    
    def _pdf(self,x,mean,var):
        return 1.0/sqrt(2*pi*var)*exp(((x-mean)**2)/(-2*var))
        
        
    def transform(self,Xtest,prior0,prior1):
        proj=np.dot(self.w, Xtest.T).T
        return proj

In [10]:
F=Fld()
fld_train=F.fit(Xtrain,ytrain)
fld_test=F.transform(Xtest,p[0],p[1])

In [11]:

#old code for euclidean distance
def Euclid(rain_data,est_data,prior1,prior2):

    class1 = rain_data[np.where(rain_data[:,-1] == 0)]
    class2 = rain_data[np.where(rain_data[:,-1] == 1)]

    mean1=np.mean(class1[:,:-1],axis=0)
    mean2=np.mean(class2[:,:-1],axis=0)
    std1=np.std(class1[:,:-1],axis=0)
    std2=np.std(class2[:,:-1],axis=0)
    data_wc=est_data[:,:-1]

    std1=std1.T
    std2=std2.T
    std_ave=(std1+std2)/2
    std_ave=np.sum(std_ave)/std_ave.shape[0]
#    print("stdev: ",std_ave)

    mean1=mean1.T
#    print("mean1: ",mean1)
    mean2=mean2.T
 #   print("mean2: ",mean2)
    norm_class1=-(np.linalg.norm(est_data[:,0:-1]-mean1, axis=1))**2/std_ave**2+np.log(prior1)
    norm_class2=-(np.linalg.norm(est_data[:,0:-1]-mean2,axis=1))**2/std_ave**2+np.log(prior2)
    ans=[0 if x > y else 1 for (x,y) in zip(norm_class1, norm_class2)]
    
    count=[1 if x==y else 0 for (x,y) in zip(est_data[:,-1],ans)]
    count=np.sum(count)
    total=est_data.shape[0]
    acc=count/total
    count_class1=np.sum([1 if (x==0 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class1=np.sum([1 if x==0 else 0 for x in est_data[:,-1:]])
    acc_class1=count_class1/total_class1
    count_class2=np.sum([1 if (x==1 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class2=np.sum([1 if x==1 else 0 for x in est_data[:,-1:]])
    acc_class2=count_class2/total_class2
    return(acc,acc_class1,acc_class2,ans)

#old code for mahalanobis
def Mahalanobis(rain_data,est_data,prior1,prior2):
    class1 = rain_data[np.where(rain_data[:,-1] == 0)]
    class2 = rain_data[np.where(rain_data[:,-1] == 1)]

    mean1=np.mean(class1[:,:-1],axis=0)
    mean2=np.mean(class2[:,:-1],axis=0)
  #  print(mean1)
   # print(mean2)
    
    cov1=np.cov(class1[:,:-1].T)
    cov2=np.cov(class2[:,:-1].T)
    cov_ave=(cov1+cov2)/2
   # print("MLS: ", cov_ave)
    mean1=mean1.T
    mean2=mean2.T
  #  print(mean1)
  #  print(mean2)
    n=est_data.shape[0]
  #  print(n)
    a1=est_data[:,0:-1]-mean1
    b1=(1.0/cov_ave)
   # print("inv: ", b1)
   # print("inv_trans: ", b1.T)
    c1=est_data[:,0:-1]-mean1
    dist1=np.dot(a1,np.dot(b1,c1.T))
   # print(dist1.shape)
    dist1=-.5*np.sum(dist1,axis=1)+np.log(prior1)
   # print(dist1.shape)
    a2=est_data[:,0:-1]-mean2
    b2=(1.0/cov_ave)
    c2=est_data[:,0:-1]-mean2
    dist2=np.dot(a2,np.dot(b2,c2.T))
    dist2=-.5*np.sum(dist2,axis=1)+np.log(prior2)
    
    ans=[0 if x > y else 1 for (x,y) in zip(dist1, dist2)]
    count=[1 if x==y else 0 for (x,y) in zip(est_data[:,-1],ans)]
    count=np.sum(count)
    total=est_data.shape[0]
    acc=count/total
    count_class1=np.sum([1 if (x==0 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class1=np.sum([1 if x==0 else 0 for x in est_data[:,-1:]])
    acc_class1=count_class1/total_class1
    count_class2=np.sum([1 if (x==1 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class2=np.sum([1 if x==1 else 0 for x in est_data[:,-1:]])
    acc_class2=count_class2/total_class2
    return (acc,acc_class1,acc_class2,ans)
    
#old code for quadratic distance
def Quadratic(rain_data,est_data,prior1,prior2):
    class1 = rain_data[np.where(rain_data[:,-1] == 0)]
    class2 = rain_data[np.where(rain_data[:,-1] == 1)]

    mean1=np.mean(class1[:,:-1],axis=0)
    mean2=np.mean(class2[:,:-1],axis=0)
    cov1=np.cov(class1[:,:-1].T)
    cov2=np.cov(class2[:,:-1].T)
    mean1=mean1.T
    mean2=mean2.T
   # print("mean1: ", mean1)
   # print("mean2: ", mean2)
    n=est_data.shape[0]
    a1=est_data[:,0:-1]-mean1
    b1=1.0/(cov1)
    #print("inverse sigma 1: ", b1)
    c1=est_data[:,0:-1]-mean1
    dist1=np.dot(a1,np.dot(b1,c1.T))
    dist1=-.5*np.sum(dist1,axis=1)/(n-1)-.5*np.log((cov1))+np.log(prior1)
    
    a2=est_data[:,0:-1]-mean2
    b2=1.0/(cov2)
    #print("inverse sigma 2: ", b2)
    
    c2=est_data[:,0:-1]-mean2
    dist2=np.dot(a2,np.dot(b2,c2.T))
    dist2=-.5*np.sum(dist2,axis=1)/(n-1)-.5*np.log((cov2))+np.log(prior2)
    #print("det1: ", -.5*np.log(np.linalg.det(cov1)))
   # print("det2: ", .5*np.log(np.linalg.det(cov2)))
    ans=[0 if x > y else 1 for (x,y) in zip(dist1, dist2)]
    count=[1 if x==y else 0 for (x,y) in zip(est_data[:,-1],ans)]
    count=np.sum(count)
    total=est_data.shape[0]
    acc=count/total
    count_class1=np.sum([1 if (x==0 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class1=np.sum([1 if x==0 else 0 for x in est_data[:,-1:]])
    acc_class1=count_class1/total_class1
    count_class2=np.sum([1 if (x==1 and x==y) else 0 for (x,y) in zip(est_data[:,-1],ans)])
    total_class2=np.sum([1 if x==1 else 0 for x in est_data[:,-1:]])
    acc_class2=count_class2/total_class2
    return (acc,acc_class1,acc_class2,ans)

In [12]:
t0 = time.time()           # start time
acc,ac0,ac1,ans = Mahalanobis(np.c_[fld_train, ytrain], np.c_[fld_test,ytest], p[0],p[1])
t1 = time.time()           # ending time
print(acc, ac0, ac1, t1-t0)

0.475 0.5088888888888888 0.37333333333333335 0.005984306335449219


In [13]:
t0 = time.time()           # start time
acc,ac0,ac1,ans = Euclid(np.c_[fld_train, ytrain], np.c_[fld_test,ytest], p[0],p[1])
t1 = time.time()           # ending time
print(acc, ac0, ac1, t1-t0)

0.47333333333333333 0.5044444444444445 0.38 0.005013942718505859


In [14]:
t0 = time.time()           # start time
acc,ac0,ac1,ans = Quadratic(np.c_[fld_train, ytrain], np.c_[fld_test,ytest], p[0],p[1])
t1 = time.time()           # ending time
print(acc, ac0, ac1, t1-t0)

0.475 0.5088888888888888 0.37333333333333335 0.006979703903198242
